In [2]:
! pip install faiss-cpu

   ---------------------------------------- 0.0/18.2 MB ? eta -:--:--
   -------- ------------------------------- 3.9/18.2 MB 21.0 MB/s eta 0:00:01
   ------------------ --------------------- 8.4/18.2 MB 20.6 MB/s eta 0:00:01
   ------------------------------- -------- 14.4/18.2 MB 23.7 MB/s eta 0:00:01
   ---------------------------------------- 18.2/18.2 MB 23.6 MB/s  0:00:00


In [3]:
from mistralai import Mistral
import requests
import numpy as np
import faiss
import os
from getpass import getpass

# api_key= getpass("Type your API Key")
api_key = os.getenv("MISTRAL_API_KEY")
client = Mistral(api_key=api_key)

In [4]:
f = open('sample_data/paul_graham_essay.txt', 'r')
essay = f.read()
f.close()

In [5]:
len(essay)

75039

In [6]:
# split the document(essay) into chunks 
chunk_size = 2048
chunks = [essay[i:i+chunk_size] for i in range(0, len(essay), chunk_size)]

In [7]:
len(chunks)

37

In [12]:
# create embeddings for each chunk
def get_text_embedding(input):
    embeddings_batch_response = client.embeddings.create(
        model="mistral-embed",
        inputs = input       
    )
    return embeddings_batch_response.data[0].embedding

In [13]:
text_embeddings = np.array([get_text_embedding(chunk) for chunk in chunks]).astype("float32")

In [14]:
text_embeddings.shape

(37, 1024)

In [15]:
text_embeddings[0]

array([-0.03839111,  0.07592773,  0.00132561, ..., -0.01730347,
       -0.01808167, -0.00711441], shape=(1024,), dtype=float32)

In [16]:
# storing the text embeddings in vector database
d = text_embeddings.shape[1]  # dimension
index = faiss.IndexFlatL2(d)  # build the index
index.add(text_embeddings)  # add vectors to the index

In [17]:
question = "What were the two main things the author worked on before college?"
question_embeddings = np.array([get_text_embedding(question)])
question_embeddings.shape

(1, 1024)

In [18]:
type(question_embeddings)

numpy.ndarray

In [19]:
# retrive top 3 similar chunks from the vector database
D, I = index.search(question_embeddings.astype("float32"), k=3)  # actual search
print(I)

[[ 0  3 11]]


In [ ]:
# D is the dimension of the text embeddings. It represents the number of features in each embedding vector. In this case, it is obtained from the shape of the text_embeddings array, specifically the second dimension (number of columns) which corresponds to the size of each embedding vector.
# I is a 2D array containing the indices of the top k (in this case, 3) nearest neighbor vectors in the index for each query vector. Each row corresponds to a query vector, and each column contains the index of one of the nearest neighbors in the original dataset.

In [21]:
retrieved_chunks = [chunks[i] for i in I[0]]

In [23]:
# retrieved_chunks

In [24]:
# answer the user query using the retrieved chunks as context
context = "\n".join(retrieved_chunks)
prompt = f"Answer the question based on the context below.\n\nContext: {context}\n\nQuestion: {question}\n\nAnswer:"


def run_mistral(user_message,model="mistral-tiny"):
    message = [
        {
            'role':'user',
            'content':user_message
        }
    ]
    chat_response = client.chat.complete(
        model=model,
        messages = message
    )
    return chat_response.choices[0].message.content

In [25]:
run_mistral(prompt)

'The two main things the author worked on before college were writing short stories and programming on the IBM 1401 computer.'